# CourseData ETL

## STEP 1: Load SQL and create CourseData database

In [2]:
%load_ext sql
import pandas as pd
import sqlite3

In [3]:
%sql sqlite:///DBTest.db
conn = sqlite3.connect('DBTest.db')

## STEP 2: Create tables as designed in Lucidchart

In [4]:
%%sql

DROP TABLE IF EXISTS PROFESSORS;
CREATE TABLE PROFESSORS (
    Professor_id INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [5]:
%%sql
DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS (
    Program_id INTEGER PRIMARY KEY,
    program_code TEXT NOT NULL,
    program_name TEXT NOT NULL
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [8]:
%%sql
DROP TABLE IF EXISTS LOCATIONS;
CREATE TABLE LOCATIONS (
    Location_id INTEGER PRIMARY KEY,
    location TEXT NOT NULL
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [7]:
%%sql
DROP TABLE IF EXISTS COURSES;
CREATE TABLE COURSES (
    Course_id INTEGER PRIMARY KEY, 
    CatalogYear TEXT NOT NULL,
    Catalog_id TEXT NOT NULL,
    Title TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Attributes TEXT,
    Prereqs TEXT,
    Coreqs TEXT,
    Description TEXT,
    Fee TEXT,
    Program_id INTEGER NOT NULL,
    FOREIGN KEY (Program_id) REFERENCES PROGRAMS(Program_id)
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [60]:
%%sql
DROP TABLE IF EXISTS COURSE_OFFERINGS;
CREATE TABLE COURSE_OFFERINGS (
    Offering_id INTEGER PRIMARY KEY,
    Term TEXT NOT NULL,
    Section TEXT NOT NULL,
    Crn INTEGER NOT NULL,
    Cap INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL,
    Timecodes TEXT,
    Course_id INTEGER NOT NULL,
    Catalog_id TEXT,
    Program_id INTEGER,
    Name TEXT NOT NULL,
    FOREIGN KEY (Name) REFERENCES PROFESSORS(Name)
    FOREIGN KEY (Program_id) REFERENCES PROGRAMS(Program_id)
    FOREIGN KEY (Course_id) REFERENCES COURSES(Course_id)
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [38]:
%%sql
DROP TABLE IF EXISTS MEETINGS;
CREATE TABLE MEETINGS (
    Meetings_id INTEGER PRIMARY KEY,
    Day TEXT,
    Start TEXT NOT NULL,
    End TEXT NOT NULL,
    Location_id INTEGER NOT NULL,
    Offering_id INTEGER,
    FOREIGN KEY (Location_id) REFERENCES LOCATIONS(Location_id),
    FOREIGN KEY (Offering_id) REFERENCES COURSE_OFFERINGS(Offering_id)
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [9]:
%%sql
DROP TABLE IF EXISTS YEAR;
CREATE TABLE YEAR (
    Year TEXT NOT NULL,
    Term TEXT
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [10]:
%%sql
INSERT INTO YEAR (Term, Year) VALUES 
('Fall2017','2017_2018'),
('Winter2018','2017_2018'),
('Spring2018','2017_2018'),
('Summer2018','2017_2018'),
('Fall2018','2018_2019'),
('Winter2019','2018_2019'),
('Spring2019','2018_2019');

 * sqlite:///DBTest.db
7 rows affected.


[]

In [11]:
%%sql
SELECT * FROM YEAR
LIMIT (2);

 * sqlite:///DBTest.db
Done.


Year,Term
2017_2018,Fall2017
2017_2018,Winter2018


## STEP 3: Import CSV files

In [12]:
# importing course catalog csvs (taken from professsor's example )
course_catalog = ['2017_2018', '2018_2019']

# loop using pandas
for cat_year in course_catalog:
    path = 'SourceData/Catalogs/CourseCatalog'+cat_year+'.csv' # path to directory
    data = pd.read_csv(path) # read all csvs in path
    data['cat_year'] = cat_year # add new column 'Term' to differentiate between 2017-18, 2018-19
    data.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False) # converting to sql & unique

In [13]:
# meetings df

semesters = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018','Spring2015', 'Spring2016','Spring2017','Spring2018','Spring2019','SpringBreak2017',
                   'Summer2015', 'Summer2016','Summer2017','Summer2018','Winter2015','Winter2016', 'Winter2017','Winter2018']

for semester in semesters: 
    path = 'SourceData/'+semester+'/course_meetings.csv'
    data = pd.read_csv(path)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False) 

In [14]:
# courses df
semesters = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018','Spring2015', 'Spring2016','Spring2017','Spring2018','Spring2019','SpringBreak2017',
                   'Summer2015', 'Summer2016','Summer2017','Summer2018','Winter2015','Winter2016', 'Winter2017','Winter2018']

for semester in semesters:
    path = 'SourceData/'+semester+'/courses.csv'
    data = pd.read_csv(path)
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 

In [15]:
%%sql
--check for duplication
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_OFFERINGS) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_OFFERINGS)) as 'DistinctCount';

 * sqlite:///DBTest.db
Done.


Count,DistinctCount
15937,15937


In [16]:
%%sql
-- check for duplication in course meetings

SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'DistinctCount';

 * sqlite:///DBTest.db
Done.


Count,DistinctCount
284907,284847


In [17]:
%%sql
--check for duplication in catalog courses

SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOG_COURSES) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOG_COURSES)) as 'DistinctCount';

 * sqlite:///DBTest.db
Done.


Count,DistinctCount
4440,4440


## STEP 4: POPULATING THE TABLES

In [18]:
%%sql
DELETE FROM PROFESSORS;
INSERT INTO PROFESSORS(Name)
    SELECT DISTINCT primary_instructor
    FROM IMPORT_COURSE_OFFERINGS;
SELECT * FROM PROFESSORS LIMIT 5;

 * sqlite:///DBTest.db
0 rows affected.
1104 rows affected.
Done.


Professor_id,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos


In [19]:
%%sql
DELETE FROM PROGRAMS;
INSERT INTO PROGRAMS(program_code, program_name)
    SELECT DISTINCT program_code, program_name
    FROM IMPORT_CATALOG_COURSES;
SELECT * FROM PROGRAMS LIMIT 5;

 * sqlite:///DBTest.db
0 rows affected.
83 rows affected.
Done.


Program_id,program_code,program_name
1,AN,Asian Studies
2,BU,Business
3,BL,Black Studies
4,BEN,Bioengineering
5,AR,Arabic


In [5]:
%%sql
DELETE FROM COURSES;
INSERT INTO COURSES(CatalogYear, Catalog_id, Title, Credits, Attributes, Prereqs, Coreqs, Description, Fee, Program_id)
    SELECT DISTINCT cat_year, Catalog_id, course_title, credits, attributes, prereqs, coreqs, description, fees, Program_id 
    FROM IMPORT_CATALOG_COURSES
        JOIN PROGRAMS USING (program_code);
SELECT * FROM COURSES
LIMIT 5;

 * sqlite:///DBTest.db
4440 rows affected.
4440 rows affected.
Done.


Course_id,CatalogYear,Catalog_id,Title,Credits,Attributes,Prereqs,Coreqs,Description,Fee,Program_id
1,2017_2018,AN 0301,Independent Study,1-3 Credits,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,None,1
2,2017_2018,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",None,1
3,2017_2018,BU 0211,Legal Environment of Business,3 Credits,None,Junior standing.,None,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets.",None,2
4,2017_2018,BU 0220,Environmental Law and Policy,3 Credits,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective",None,None,"This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights.",None,2
5,2017_2018,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,None,BU 0211.,None,"This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments.",None,2


In [22]:
%%sql
DELETE FROM LOCATIONS;
INSERT INTO LOCATIONS(location)
    SELECT DISTINCT location
    FROM IMPORT_COURSE_MEETINGS
    ORDER BY location;
SELECT * FROM LOCATIONS LIMIT 5;

 * sqlite:///DBTest.db
0 rows affected.
207 rows affected.
Done.


Location_id,location
1,BCC 200
2,BD
3,BH
4,BH BY ARR
5,BLM 112


In [9]:
%%sql
DELETE FROM COURSE_OFFERINGS;
INSERT INTO COURSE_OFFERINGS (Cap, Section, Crn, Actual, Remaining, Timecodes, Course_id, Name)
    SELECT DISTINCT cap, section, crn, act, rem, timecodes, Course_id, primary_instructor 
    FROM IMPORT_COURSE_OFFERINGS
        JOIN COURSES ON (IMPORT_COURSE_OFFERINGS.catalog_id = COURSES.Catalog_id)
        JOIN PROGRAMS USING (Program_id) 
        JOIN PROFESSORS ON (IMPORT_COURSE_OFFERINGS.Primary_instructor = PROFESSORS.Name);
SELECT DISTINCT * FROM COURSE_OFFERINGS
LIMIT 50;

 * sqlite:///DBTest.db
30486 rows affected.
30486 rows affected.
Done.


Offering_id,Term,Section,Crn,Cap,Actual,Remaining,Timecodes,Course_id,Catalog_id,Program_id,Name
1,None,DL,34130,1,0,1,[],1,None,None,Danke Li
2,None,PL,39186,0,1,-1,[],1,None,None,Philip J. Lane
3,None,A,71858,0,0,0,[],1,None,None,TBA
4,None,JX,73018,0,1,-1,[],1,None,None,Jiwei Xiao
5,None,DL,73057,0,1,-1,[],1,None,None,Danke Li
6,None,AA,77054,0,1,-1,[],1,None,None,Anna-Maria Aksan
7,None,JX,79199,0,2,-2,[],1,None,None,Jiwei Xiao
8,None,01,32874,3,1,2,[],2,None,None,Ive E. Covaci
9,None,B,34010,5,2,3,['W 0630pm-0900pm 01/20-04/30 CNS 305'],2,None,None,Jiwei Xiao
10,None,01,35347,3,2,1,[],2,None,None,Danke Li


In [40]:
%%sql
DELETE FROM MEETINGS;
INSERT INTO MEETINGS(Meetings, Day, Start, End, Location_id)
    SELECT DISTINCT Meetings, day, start, end, Location_id
    FROM IMPORT_CATALOG_COURSES
        JOIN COURSE_OFFERINGS USING (Term, Crn) 
        JOIN LOCATIONS ON (LOCATIONS.location_id = IMPORT_CATALOG_COURSES.location);
SELECT * FROM MEETINGS LIMIT 5;  

 * sqlite:///DBTest.db
0 rows affected.
(sqlite3.OperationalError) table MEETINGS has no column named Meetings
[SQL: INSERT INTO MEETINGS(Meetings, Day, Start, End, Location_id)
    SELECT DISTINCT meetings, day, start, end, Location_id
    FROM IMPORT_CATALOG_COURSES
        JOIN COURSE_OFFERINGS USING (Term, Crn) 
        JOIN LOCATIONS ON (LOCATIONS.location_id = IMPORT_CATALOG_COURSES.location);]
(Background on this error at: http://sqlalche.me/e/e3q8)
